In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from dataloading.nvidia import NvidiaResizeAndCrop, Normalize, NvidiaDataset
from network import PilotNet
from trainer import Trainer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Datasets

In [13]:
root_path = Path("/home/romet/data/datasets/ut/nvidia-data")

train_paths = [root_path / "2021-05-20-12-36-10_e2e_sulaoja_20_30",
            root_path / "2021-05-20-12-43-17_e2e_sulaoja_20_30", 
            root_path / "2021-05-20-12-51-29_e2e_sulaoja_20_30",
            root_path / "2021-05-20-13-44-06_e2e_sulaoja_10_10",
            root_path / "2021-05-20-13-51-21_e2e_sulaoja_10_10"]

trainset = NvidiaDataset(train_paths)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True,
                                         num_workers=3, pin_memory=True, persistent_workers=True)

In [14]:
valid_paths = [root_path / "2021-05-20-13-59-00_e2e_sulaoja_10_10"]
validset = NvidiaDataset(valid_paths)
validloader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=False)

## Model

In [15]:
model = PilotNet()
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, betas=(0.9, 0.999), 
                              eps=1e-08, weight_decay=0.01, amsgrad=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

## Train

In [16]:
N_EPOCHS = 100

trainer = Trainer("models/1-pilotnet-base/1cam-v2", device)
trainer.train(model, trainloader, validloader, optimizer, criterion, N_EPOCHS)

  0%|          | 0/498 [00:00<?, ?it/s]

RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same